In [ ]:
import logging
from xbrl.cache import HttpCache
from xbrl.instance import XbrlParser, XbrlInstance

import json
import pandas as pd

In [ ]:
# just to see which files are downloaded
logging.basicConfig(level=logging.INFO)

cache: HttpCache = HttpCache("./cache")
cache.set_headers({"From": "YOUR@EMAIL.com", "User-Agent": "py-xbrl/2.1.0"})
parser = XbrlParser(cache)

schema_url = "https://www.sec.gov/Archives/edgar/data/0000320193/000032019321000105/aapl-20210925.htm"
inst: XbrlInstance = parser.parse_instance(schema_url)

In [ ]:
report = json.loads(inst.json(override_fact_ids=True))

In [ ]:
def satz_erkenner(i, leerzeichen=3):
    if len(str(i).split(" ")) > leerzeichen:
        return True
    return False

In [ ]:
for key, value in report["facts"].items():
    if "value" in value.keys():
        if isinstance(value["value"], str):
            string = report["facts"][key]["value"]
            if len(string.split(" ")) > 4:
                print(string)

In [ ]:
value

In [ ]:
df = pd.json_normalize(report["facts"]).T

In [ ]:
df = df.rename(columns={0: "value"})

In [ ]:
df["is_sentence"] = df["value"].apply(lambda x: satz_erkenner(x, 4))

In [ ]:
import re

In [ ]:
pattern = r"(?<!\b(?:Mr|Mrs|Dr|Ms)\b)(?<!\b\w)\.\s"

In [ ]:
sentences = (
    df.query("is_sentence==True")
    .reset_index(drop=True)["value"]
    .apply(lambda x: x.split(r". "))
    .explode()
    .reset_index(drop=True)
)

In [ ]:
sentences = pd.DataFrame(sentences)

In [ ]:
sentences["e_score"] = sentences["value"].apply(lambda x: len(x))
sentences["s_score"] = sentences["value"].apply(lambda x: len(x) * 2)
sentences["g_score"] = sentences["value"].apply(lambda x: len(x) * 3)

In [ ]:
sentences